<a href="https://colab.research.google.com/github/ougrid/my-knowledge-resource/blob/master/ASR.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install accelerate datasets

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 485.4/485.4 kB 9.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 7.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 143.5/143.5 kB 8.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 4.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 23.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 23.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 36.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 5.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 9.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 7.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 6.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

In [ ]:
from transformers import WhisperProcessor, WhisperForConditionalGeneration, pipeline

from datasets import load_dataset

from IPython.display import Audio
import torchaudio
import torch

In [ ]:
model_name = 'biodatlab/whisper-th-large-v3-combined'

In [ ]:
processor = WhisperProcessor.from_pretrained(model_name)
model = WhisperForConditionalGeneration.from_pretrained(
    model_name,
    torch_dtype="auto",
    device_map="auto",
)

preprocessor_config.json:   0%|          | 0.00/340 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/283k [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/836k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/2.48M [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/494k [00:00<?, ?B/s]

...

added_tokens.json:   0%|          | 0.00/34.6k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/2.19k [00:00<?, ?B/s]

config.json:   0%|          | 0.00/1.29k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/3.22G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/3.90k [00:00<?, ?B/s]

In [ ]:
dataset = load_dataset("mozilla-foundation/common_voice_15_0", "th", split="test", streaming=True, trust_remote_code = True)

sample = next(iter(dataset))

Reading metadata...: 11032it [00:00, 20657.64it/s]


In [ ]:
for item in dataset:
  print(item)
  break

Reading metadata...: 11032it [00:00, 38490.86it/s]


{'client_id': '00086f5dc46f9038f13bbd829c4118fab3ac28688d5ee878de75e0e17078ad08a06dd184b712848ce241c5f3f03501529fa67ded21999d274f56d5c67aaebd55', 'path': 'th_test_0/common_voice_th_25695281.mp3', 'audio': {'path': 'th_test_0/common_voice_th_25695281.mp3', 'array': array([-2.97785021e-23,  1.15805286e-23,  4.71492949e-23, ...,
       -1.03299681e-05, -4.60250230e-05, -3.92288675e-05]), 'sampling_rate': 48000}, 'sentence': 'ใครเป็นผู้รับ', 'up_votes': 2, 'down_votes': 0, 'age': '', 'gender': '', 'accent': '', 'locale': 'th', 'segment': '', 'variant': ''}


In [ ]:
Audio(sample["audio"]["array"], rate=sample["audio"]["sampling_rate"])

In [ ]:
resampler = torchaudio.transforms.Resample(sample["audio"]["sampling_rate"], 16000)
audio = resampler(torch.tensor(sample["audio"]["array"], dtype=torch.float32))

In [ ]:
model_inputs = processor(audio, sampling_rate=16000, return_tensors="pt").to(model.device, model.dtype)
model_inputs.keys()

dict_keys(['input_features'])

In [ ]:
generated_ids = model.generate(
    **model_inputs,
    forced_decoder_ids=processor.get_decoder_prompt_ids(language="th", task="transcribe"),
    use_cache=True,
)

The attention mask is not set and cannot be inferred from input because pad token is same as eos token. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.


In [ ]:
generated_text = processor.batch_decode(generated_ids)
print(generated_text)

['ใครเป็นผู้รับ']
